# Лабораторная работа 1. Прогнозирование моделью линейной регрессии

### 0. Загрузка данных

Перед нормированием и расчетом весов необходимо загрузить данные с помощью библиотеки numpy (```X``` - матрица признаков, ```y``` - матрица целевых значений).

In [138]:
import numpy as np

from sklearn.metrics import r2_score

# получение данных из файлов
X = np.genfromtxt("./datasets/lab1_X.csv")
y = np.genfromtxt("./datasets/lab1_y.csv").reshape(X.shape[0], 1)

print(f"Матрицы X и y:\n{X}\n{y}")


Матрицы X и y:
[[7.000e+00 1.590e+01 8.200e+00 5.100e+00 1.380e+01 2.290e+02 1.720e+02
  2.000e+01 2.104e+03]
 [7.200e+00 1.820e+01 7.400e+00 6.100e+00 1.430e+01 1.460e+02 1.670e+02
  2.910e+01 2.489e+03]
 [7.900e+00 1.970e+01 6.400e+00 4.700e+00 1.980e+01 1.740e+02 1.440e+02
  2.280e+01 2.428e+03]
 [7.700e+00 2.080e+01 6.900e+00 5.200e+00 1.710e+01 1.280e+02 1.110e+02
  4.270e+01 2.494e+03]
 [9.200e+00 1.590e+01 7.800e+00 5.300e+00 1.670e+01 1.690e+02 1.480e+02
  2.270e+01 2.094e+03]
 [7.600e+00 1.640e+01 6.700e+00 4.700e+00 1.550e+01 1.440e+02 1.500e+02
  2.790e+01 1.768e+03]
 [7.300e+00 1.830e+01 6.300e+00 4.900e+00 1.960e+01 1.380e+02 1.330e+02
  3.370e+01 1.982e+03]
 [7.900e+00 1.640e+01 6.800e+00 5.000e+00 1.760e+01 1.970e+02 1.550e+02
  2.660e+01 1.621e+03]
 [7.900e+00 1.700e+01 6.300e+00 4.400e+00 2.010e+01 1.820e+02 1.590e+02
  3.050e+01 1.631e+03]
 [8.000e+00 1.690e+01 8.200e+00 4.600e+00 1.550e+01 5.200e+02 1.970e+02
  1.910e+01 1.066e+03]
 [7.200e+00 1.760e+01 8.100e+00 5.2

Разделим все данные на выборку для обучения и выборку для валидации (```X_train```, ```y_train``` и ```X_test```, ```y_test``` соответственно)

In [139]:
# разделяем данные так: 70% для обучения, 30% для валидации
split = round(X.shape[0] * 0.7)

# Разделение для матрицы признаков
X_train = X[:split]
X_test = X[split:]

# Разделение для матрицы целевых значений
y_train = y[:split]
y_test = y[split:]

print(f"Размерности матриц: X_test - {X_test.shape}, X_train - {X_train.shape}, y_test - {y_test.shape}, y_train - {y_train.shape}")

Размерности матриц: X_test - (15, 9), X_train - (35, 9), y_test - (15, 1), y_train - (35, 1)


### 1. Нормирование (масштабирование) исходных данных.

Теперь необходимо нормализовать данные, поскольку разброс значенией данных слишком большой. Будет использоваться Z-нормализация

In [140]:
# функции для нахождения среднего и стандартного распределения
X_train = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test = (X_test - np.mean(X_test, axis=0)) / np.std(X_test, axis=0)

print(X_train)

[[-0.86580164  0.12296253  1.55757652  0.91545987 -1.66258049  0.94244347
   0.9941688  -1.24316968  0.76671086]
 [-0.79287009  0.94101901  0.35944073  2.02032522 -1.43859928 -0.27652282
   0.80669085 -0.26936362  1.51428169]
 [-0.53760968  1.4745341  -1.13822899  0.47351372  1.02519397  0.13469473
  -0.05570773 -0.94353704  1.39583541]
 [-0.61054123  1.86577851 -0.38939413  1.0259464  -0.18430453 -0.54087695
  -1.29306223  1.18599488  1.52399041]
 [-0.06355463  0.12296253  0.95850863  1.13643294 -0.3634895   0.06126302
   0.09427463 -0.95423821  0.74729344]
 [-0.647007    0.30080089 -0.68892807  0.47351372 -0.90104439 -0.3058955
   0.16926581 -0.39777761  0.11428541]
 [-0.75640432  0.97658668 -1.28799597  0.69448679  0.93560149 -0.39401355
  -0.46815923  0.22288999  0.52981829]
 [-0.53760968  0.30080089 -0.5391611   0.80497333  0.03967667  0.47248057
   0.35674376 -0.53689276 -0.17115072]
 [-0.53760968  0.51420693 -1.28799597  0.14205412  1.1595827   0.25218545
   0.50672613 -0.119547

Также нужно добавить единичный столбец к данным, чтобы был свободный параметр ```w0```.

In [141]:
X_train = np.append(X_train, np.ones(X_train.shape[0]).reshape((X_train.shape[0], 1)), axis=1)
X_test = np.append(X_test, np.ones(X_test.shape[0]).reshape((X_test.shape[0], 1)), axis=1)

print(X_train)

[[-0.86580164  0.12296253  1.55757652  0.91545987 -1.66258049  0.94244347
   0.9941688  -1.24316968  0.76671086  1.        ]
 [-0.79287009  0.94101901  0.35944073  2.02032522 -1.43859928 -0.27652282
   0.80669085 -0.26936362  1.51428169  1.        ]
 [-0.53760968  1.4745341  -1.13822899  0.47351372  1.02519397  0.13469473
  -0.05570773 -0.94353704  1.39583541  1.        ]
 [-0.61054123  1.86577851 -0.38939413  1.0259464  -0.18430453 -0.54087695
  -1.29306223  1.18599488  1.52399041  1.        ]
 [-0.06355463  0.12296253  0.95850863  1.13643294 -0.3634895   0.06126302
   0.09427463 -0.95423821  0.74729344  1.        ]
 [-0.647007    0.30080089 -0.68892807  0.47351372 -0.90104439 -0.3058955
   0.16926581 -0.39777761  0.11428541  1.        ]
 [-0.75640432  0.97658668 -1.28799597  0.69448679  0.93560149 -0.39401355
  -0.46815923  0.22288999  0.52981829  1.        ]
 [-0.53760968  0.30080089 -0.5391611   0.80497333  0.03967667  0.47248057
   0.35674376 -0.53689276 -0.17115072  1.        ]
 

### 2. Расчет весов линейной регрессии по аналитической формуле.

После нормализации можно рассчитать веса с помощью аналитической формулы (здесь также будут использоваться функции numpy)

In [142]:
cov_mat = np.matmul(np.transpose(X_train), X_train) # получение ковариационной матрицы
temp = np.matmul(np.linalg.inv(cov_mat), np.transpose(X_train)) # произведение обратной ковариационной матрицы и X транспонированной
w = np.matmul(temp, y_train) # получение матрицы весов

print(f"Полученные веса:\n{w}")

Полученные веса:
[[-1.56018400e-01]
 [-2.23529642e+00]
 [ 6.68635399e-01]
 [-8.35658594e-03]
 [ 4.98044131e-01]
 [ 3.28450435e-01]
 [-6.46361963e-01]
 [ 1.75903141e-01]
 [ 7.07629979e-01]
 [ 5.92542857e+01]]


### 3. Построение и интепретация корреляционной матрицы. Определение степени мультиколлинеарности на основе числа обусловленности.

Теперь определим степень мультиколлинеарности на основе числа обусловленности. Будет использоваться функция ```eigvals``` из linalg для подсчета собственных значений.

In [143]:
print(f"Ковариационная матрица:\n{cov_mat}")
print(f"Число обусловленности: {np.linalg.cond(X_train, p = 2)}")

Ковариационная матрица:
[[ 3.50000000e+01 -2.98986542e+01 -4.07418083e-01 -2.83018361e+01
   2.31475804e+00 -1.14787075e+01 -2.20240523e+01  1.81055400e+01
  -2.13634138e+01  7.16093851e-15]
 [-2.98986542e+01  3.50000000e+01 -1.96774309e+00  2.76985550e+01
   9.36586120e-01  1.03188134e+01  1.89319196e+01 -1.90080387e+01
   2.10393130e+01  1.17822418e-14]
 [-4.07418083e-01 -1.96774309e+00  3.50000000e+01  1.16823472e+01
  -1.52562601e+00  1.04072905e+01  1.01574993e+01 -5.58694650e-01
  -1.06026011e+01  3.79696274e-14]
 [-2.83018361e+01  2.76985550e+01  1.16823472e+01  3.50000000e+01
  -2.82044039e+00  1.00640950e+01  2.05847723e+01 -1.51980680e+01
   1.78342877e+01  4.32986980e-15]
 [ 2.31475804e+00  9.36586120e-01 -1.52562601e+00 -2.82044039e+00
   3.50000000e+01 -6.13768931e+00  5.69501235e-01  1.05578393e+01
   2.69406736e+00 -3.91908728e-14]
 [-1.14787075e+01  1.03188134e+01  1.04072905e+01  1.00640950e+01
  -6.13768931e+00  3.50000000e+01  2.10598920e+01 -1.97868751e+01
  -2.6775

### 4. Анализ регрессионных остатков.

Найдем среднеквадратическую ошибку (MSE) и коэффициент детерминации (r2) для матрицы ```X_train```, будем искать регрессионные остатки относительно ```y_train```.

In [144]:
# оценочные значения целевого набора обучающих данных
y_pred_train = np.matmul(X_train, w)

# MSE для обучающий данных
Eps_train = y_pred_train - y_train
MSE_train = np.matmul(np.transpose(Eps_train), Eps_train) / X_train.shape[0]

# коэффициент детерминации для обучающих данных
r2_train = r2_score(y_train, y_pred_train)

print(f"MSE: {MSE_train[0][0]}\nR2: {r2_train}")

MSE: 2.4463814111475926
R2: 0.6766573558026336


Теперь найдем MSE для валидационной выборки (```X_test``` и ```y_test```).

In [150]:
# оценочные значения целевого набора данных для валидации
y_pred_test = np.matmul(X_test, w)

# MSE для валидационных данных
Eps_test = y_pred_test - y_test
MSE_test = np.matmul(np.transpose(Eps_test), Eps_test) / X_test.shape[0]

print(f"MSE: {MSE_test[0][0]}")

MSE: 20.389900510037812


### 5. Определение весов линейной регрессии градиентным методом. Проанализировать изменение ошибки от итерации к итерации

Сначала инициализурем весы (каждый параметр будет равен единице).

In [146]:
# 1. Инициализация весов
w = np.ones((X_train.shape[1], 1))

print(w)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]


Теперь циклично будем расчитывать оценочные значения по весам, градиент функции потерь и новые значения весов. Эти шаги будут выполняться на протяжении 500 итераций алгоритма.

In [147]:
learning_rate = 0.1
S = []

for i in range(500):
    # 2. Расчет предсказанного значения y_pred по весам w
    y_pred = np.matmul(X_train, w)
    Eps_train = y_train - y_pred

    # 3. Расчет ошибки и градиента функции потерь
    S.append((1 / X_train.shape[0]) * np.matmul(np.transpose(Eps_train), Eps_train)[0][0])

    # 4. Установка новых значений весов
    dS = (-2 / X_train.shape[0]) * np.transpose(np.matmul(np.transpose(Eps_train), X_train))
    w -= learning_rate * dS

[print(f"Эпоха {_ + 1}: {S[_]}") for _ in range(len(S))]

Эпоха 1: 3414.5719234274197
Эпоха 2: 2177.29186039666
Эпоха 3: 1394.4373505344688
Эпоха 4: 893.6640155037337
Эпоха 5: 573.1673431870605
Эпоха 6: 368.0374233041753
Эпоха 7: 236.74062770154924
Эпоха 8: 152.6966339480131
Эпоха 9: 98.89458277709178
Эпоха 10: 64.44780696269203
Эпоха 11: 42.38897633708126
Эпоха 12: 28.259054744532783
Эпоха 13: 19.204268989626303
Эпоха 14: 13.398189872953749
Эпоха 15: 9.67187602729905
Эпоха 16: 7.277172784013111
Эпоха 17: 5.735227264690987
Эпоха 18: 4.739540120024771
Эпоха 19: 4.093919718892148
Эпоха 20: 3.672773330449116
Эпоха 21: 3.3956936575330694
Эпоха 22: 3.2111942171825314
Эпоха 23: 3.0862998906177377
Эпоха 24: 2.999884737833142
Эпоха 25: 2.938408045290725
Эпоха 26: 2.893185241697878
Эпоха 27: 2.8586411361787847
Эпоха 28: 2.8311918533602167
Эпоха 29: 2.808529146481259
Эпоха 30: 2.789162244565751
Эпоха 31: 2.772124532978606
Эпоха 32: 2.7567857380693908
Эпоха 33: 2.7427316441712755
Эпоха 34: 2.729687040054409
Эпоха 35: 2.717466340030466
Эпоха 36: 2.705941

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

После использования градиентного спуска можно расчитать MSE для набора тестовых данных ```X_test``` и ```y_test```.

In [148]:
Eps_test = y_test - np.matmul(X_test, w) # регресионный остатки
MSE_test = (1 / X_test.shape[0]) * np.matmul(np.transpose(Eps_test), Eps_test) # среднеквадратичная ошибка

print(f"MSE: {MSE_test[0][0]}")

MSE: 20.389900510037812


Также найдем MSE и R2 для обучающих данных ```X_train``` и ```y_train```.

In [152]:
y_pred_train = np.matmul(X_train, w)

Eps_train = y_train - y_pred_train
MSE_train = (1 / X_train.shape[0]) * np.matmul(np.transpose(Eps_train), Eps_train)

r2_train = r2_score(y_train, y_pred_train)

print(f"MSE: {MSE_train[0][0]}\nR2: {r2_train}")

MSE: 2.4463814165668283
R2: 0.6766573550863635


### 6. Сравнение результатов по аналитическому и градиентному методу.

Если сравнивать ошибки на тестовых данных по аналитическому и градиентному методу, то можно увидеть, что результаты для первого и второго методов примерно одинаковые.

### 7. С помощью библиотеки sklearn сделать fit-predict модели линейной регрессии. Сравнить результаты с ранее полученными.

Теперь создадим модель через scikit-learn. Для этого воспользуемся одноименной библиотеки и функцией ```LinearRegression```.

In [149]:
from sklearn.linear_model import LinearRegression

# инициализация модели
model = LinearRegression()

# обучение модели
model.fit(X_train, y_train)

# получение оценочных значений
y_pred = model.predict(X_test)

# регрессионные остатки и MSE
Eps_test = y_test - y_pred
MSE_test = (1 / X_test.shape[0]) * np.matmul(np.transpose(Eps_test), Eps_test)

print(f"MSE: {MSE_test[0][0]}")

MSE: 20.389912982292394


Для полученной модели найдем MSE и R2 для ```X_train``` и ```y_train```.

In [154]:
y_pred_train = model.predict(X_train)

Eps_train = y_train - y_pred_train
MSE_train = (1 / X_train.shape[0]) * np.matmul(np.transpose(Eps_train), Eps_train)

r2_train = r2_score(y_train, y_pred_train)

print(f"MSE: {MSE_train[0][0]}\nR2: {r2_train}")

MSE: 2.446381411147593
R2: 0.6766573558026336


### 8. С помощью библиотеки statmodels получить «эконометрический» результат обучения модели линейной регрессии. Проинтерпретировать все его составляющие (в т.ч. те, которые изучались только теоретически), сравнить с предыдущими результатами.

Здесь используем библиотеку statsmodel, а именно statsmodel.api. Для обучения будет использоваться метод ```fit```, а для получения отчета - ```summary```. Модель для обучения - ```OLS```, Ordinary Linear Regression (Обыкновенная линейная регрессия), которая использовалась в предыдущих заданиях.

In [155]:
import statsmodels.api as sm

# обучение модели
results = sm.OLS(y_train, X_train).fit()

# получение отчета
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.677
Model:                            OLS   Adj. R-squared:                  0.560
Method:                 Least Squares   F-statistic:                     5.813
Date:                Tue, 03 Oct 2023   Prob (F-statistic):           0.000226
Time:                        22:32:43   Log-Likelihood:                -65.319
No. Observations:                  35   AIC:                             150.6
Df Residuals:                      25   BIC:                             166.2
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.1560      0.760     -0.205      0.8

Из этого отчета можно увидеть, что коэффициент детерминации (```R-squared```) совпадает с тем, который был подсчитан для предыдущих моделей. А также, коэффициенты ```w0..w9``` примерно совпадают с теми, которые были подсчитаны аналитическим способом.

### 9. Сравнить качество получаемых моделей на основе коэффициента детерминации и MSE.

Коэффициент детерминации R2 примерно одинаковый для всех моделей и равен ```0.677```, что близко к единице и говорит о том, что модель хорошо обучена. Для обучающего набора данных среднеквадратическая ошибка равна ```2.446```, это очень немного. А для валидационного набора ошибка составила ```20.389```, что намного больше. Это произошло из-за того, что эти данные никак не участвовали в алгоритмах расчета весов.

### 10. Сделать итоговый вывод касательно причин различия в результатах при выполнении работ разными методами, а также по получаемым моделям в целом. Провести сравнительный анализ.

Результаты примерно одинаковые для всех моделей, а именно MSE и R2 для обучающих данных и MSE для валидационных данных примерно совпадают.